In [212]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

Import the geo.json file of European countries

In [213]:
#geojson file
data = json.load(open('countries.geojson','r'))

#reading drug data
can_dat = pd.read_excel('cannabis_2022.xlsx')
coc_dat = pd.read_excel('cocaine_2022.xlsx')
alc_dat = pd.read_excel('alcohol_2022.xlsx')
ecs_dat = pd.read_excel('ecstasy_2022.xlsx')
tob_dat = pd.read_excel('tobacco_2022.xlsx')
any_dat = pd.read_excel('any_illicit_2022.xlsx')
amp_dat = pd.read_excel('amphetamines_2022.xlsx')

#iso code to country name (needed to relate geojson file to drug data)
iso_trans = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Decoding_table')[2]

#a list of all 27 countries in the European Union
table = pd.read_html('https://worldpopulationreview.com/country-rankings/european-union-countries')[0]
list_eu = [item for sublist in table.values for item in sublist]

Add country name to geojson file and keep only countries in the European Union

In [214]:
#fixing iso_trans
iso_trans = iso_trans.rename(columns = {'Country name (using title case)': 'Name'})
iso_trans['Code'] = iso_trans['Code'].apply(lambda x:str(x).lower())

#changing 'Czech Republic' for 'Czechia' as the geojson file and EU data use the latter name
list_eu[list_eu.index('Czech Republic')] = 'Czechia'

#adding name and filtering for European Union
eu_countries = []
added_countries = []

for country in data['features']:
    if len(iso_trans[iso_trans['Code'] == country['properties']['cca2']]) != 0:
        name = iso_trans[iso_trans['Code'] == country['properties']['cca2']].values[:,1][0]
        country['properties']['id'] = name
        country['id'] = name
    
    if country['properties'].get('id',0) in list_eu:
        eu_countries.append(country)
        added_countries.append(country['properties']['id']) 


Filtering drug datasets to only keep the 27 EU countries

In [215]:
df_list = [can_dat,coc_dat,alc_dat,ecs_dat,tob_dat,any_dat,amp_dat]
eu_df_list =[]
for df in df_list:
    eu_df_list.append(df[df['Country'].apply(lambda x:x in list_eu)])
    
eu_df_list = keep_eu(df_list)

Merging drug datasets and renaming columns for usability in dash app

In [216]:
# delete unused columns
for i in np.arange(0,len(eu_df_list)):
    eu_df_list[i] = eu_df_list[i].drop(['Males',"Females"],axis = 1)

#give columns unique names per drug
prefix = ['can_','coc_','alc_','ecs_','tob_','any_','amp_']
for i in np.arange(0,len(eu_df_list)):
    eu_df_list[i].columns = pd.Series(eu_df_list[i].columns).apply(lambda x: prefix[i]+str(x) if x != 'Country' else x)

# merge dataframes
drug_data = pd.DataFrame({'Country':pd.Series(list_eu)})
for df in eu_df_list:
    drug_data = pd.merge(drug_data,df)


In [223]:
drug_data.head()

,Country,can_Year,can_Sample size,can_Total,coc_Year,coc_Sample size,coc_Total,alc_Year,alc_Sample size,alc_Total,...,ecs_Total,tob_Year,tob_Sample size,tob_Total,any_Year,any_Sample size,any_Total,amp_Year,amp_Sample size,amp_Total
0,Austria,2020,4650.0,3.6,2020.0,4650.0,0.3,2020.0,4650.0,77.2,...,0.1,2020.0,4650.0,26.1,2020.0,4650.0,5.1,2015.0,3477.0,0.1
1,France,2017,20665.0,6.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Malta,2013,NaN,0.4,NaN,NaN,NaN,2013.0,NaN,58.8,...,NaN,2013.0,NaN,27.4,NaN,NaN,NaN,NaN,NaN,NaN
3,Belgium,2018,3954.0,4.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2008.0,6792.0,27.7,2018.0,3954.0,5.5,NaN,NaN,NaN
4,Germany,2018,9267.0,3.0,2018.0,9267.0,0.3,2018.0,9267.0,71.0,...,0.3,2018.0,9267.0,23.3,2018.0,9267.0,3.3,2018.0,9267.0,0.5


Creating the Dash app

In [238]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Drug consumption in the European Union',style = {'font-family':'Helvetica'}),
    html.Div('Month prevalence for different substances'),
    html.Br(),
    html.Div(['Select a substance:'
        ,dcc.Dropdown(['Cannabis','Cocaine','Alcohol','Ecstasy','Tobacco','Amphetamines','Any illicit drugs'],
                          'Cannabis', id = 'drug_input')],    
            style = {'width':'200px','height':'70px'}),
    dcc.Graph(id = 'the graph'),
    html.Div('*All data extracted from the 2022 annual Statistical Bulleting published by the European Monitoring Center for Drugs and Drug Addiction')
])

@app.callback(
    Output(component_id = 'the graph', component_property = 'figure'),
    Input(component_id = 'drug_input', component_property = 'value')
)
def deploy_figure(drug_selected):
    #translating dropdown selection to appropriate values
    color_dic = {'Cannabis':'can_Total','Cocaine':'coc_Total','Alcohol':'alc_Total','Ecstasy':'ecs_Total',
                'Tobacco':'tob_Total','Any illicit drugs':'any_Total','Amphetamines':'amp_Total'}
    color = color_dic[drug_selected]
    year_dic = {'Cannabis':'can_Year','Cocaine':'coc_Year','Alcohol':'alc_Year','Ecstasy':'ecs_Year','Tobacco':'tob_Year',
               'Any illicit drugs': 'any_Year','Amphetamines':'amp_Year'}
    year = year_dic[drug_selected]
    sample_dic = {'Cannabis':'can_Sample size','Cocaine':'coc_Sample size','Alcohol':'alc_Sample size',
                  'Ecstasy': 'ecs_Sample size','Tobacco':'tob_Sample size','Any illicit drugs':'any_Sample size',
                 'Amphetamines': 'amp_Sample size'}
    sample = sample_dic[drug_selected]
#     scale_dic = {'Cannabis':'Greens','Cocaine':'Mint','Alcohol':'alc_Total','Ecstasy':'Agsunset',
#                 'Tobacco':'Oranges','Any illicit drugs':'Greys','Amphetamines':'Teal'}
    fig = px.choropleth(drug_data,locations = 'Country', geojson = eu_countries, 
                   color = color, scope = 'europe',
                   color_continuous_scale = 'OrRd',
                   locationmode = 'country names',
                   hover_name='Country',
                   hover_data = [year,sample, color],
                   labels = {color: 'Last month prevalence (%)', year: 'Year of survey'})
    fig.update_geos(fitbounds = 'locations', visible = False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig
  
app.run_server(port = 2223, debug=False)

Dash app running on http://127.0.0.1:2223/


**REFERENCES** 

geojson file: https://github.com/Stefie/geojson-world

drug data: https://www.emcdda.europa.eu/data/stats2022/gps_en
